In [2]:
import simpy
import datetime
import random
from datetime import timedelta
import time
import pandas as pd
import numpy as np
import sqlite3


#planning to implement the station also as a segment with capacity 2
n=1

segments={}
get_station = {}
#key = station1_station2
# value of primary dict = dict of segments between station1 and station2 with secondary key as segment name and value as segment object
#tracks_segment


def convert(t):
    #converts time in seconds to time in hours and minutes
    t = int(t)
    h = str(t//3600)
    if(len(h)==1):
        h = "0"+h
    m = str((t%3600)//60)
    if(len(m)==1):
        m = "0"+m
    s = str((t%3600)%60)
    if(len(s)==1):
        s = "0"+s
    return h+"."+m+"."+s

class segment:
    def __init__(self,env,name,capacity):
        self.env = env
        #nam of the segment 
        #train_id of train present in the segment
        self.train_present = 0
        self.resource = simpy.Resource(env, capacity)
        self.name = name
        #this is the signal at the start which indicates whether incoming train should enter or not
        #assuming green signal is 1 and red signal is -1
        #initially , green is enter and red is dont enter
        #this signal is at the start of the segment
        #green if segment is free and red if segment is occupied
        self.signal = 'green' #assuming only 2 signals 'green and red
        # self.length = 1 a standard length of 1 km is assumed for each segment

    def set_signal(self,signal):
        self.signal = signal
    

class timetable:
    def __init__(self, env, l,strt_station):
        #('TBM', 'Tambaram', '16.03', '16.05', '2', '31')
        self.env = env
        # self.train_id = train_id
        self.pos = 0
        self.start_time =0
        self.strt_station = get_station[strt_station]
        self.l = l
        # l = [(station,travel_time,service_time,cummulative_distance),()]
        # each row in the scraped timetable is the tuple (station code, station name, arrival time, depart time , waiting time, cummulative distance)
        #timtable is a list of such tuples
    
    def make_schedule(self):
        l = self.l
        strt_station = self.strt_station
        for i in range(1,len(l)):
            end_station = get_station[l[i][0]]
            end_station_name = end_station.station_id
            strt_station_name = strt_station.station_id
            s = strt_station_name +"_"+ end_station_name
            if s in segments:
                #print(f"{s} segment already exists")
                # print(s)
                continue
            d = l[i][3]
            seg_list = {}
            for j in range(d):
                name = s+"_"+str(j)
                t = segment(env,name,1)
                seg_list[name]=t
            segments[s] = seg_list
            strt_station = end_station

class station:
    def __init__(self, env, station_id, platforms):
        self.env = env
        self.station_id = station_id
        self.capacity = platforms
        self.segment = segment(env,station_id,platforms)
        #has signals like segments
        #green if occupancy is less than capacity and red if occupancy is equal to capacity


    def run(self):
        while self.env.now < 200:
            if self.resource.count == self.capacity:
                print(f'Station {self.station_id} has {self.resource.count} trains at time {self.env.now}.')
            yield self.env.timeout(1)

class Train:
    def __init__(self, env, train_id,schedule):
        self.env = env
        self.train_id = train_id
        self.time_table = schedule
        self.seg_prsnt = 0
        #print("timetable of train",train_id ,"is",schedule.l , "starting from station",schedule.strt_station)
        

    def run(self):

        prev_time=0
        for i in range(len(self.time_table.l)-1):
            #ith iteration - service at ith station and travel to (i+1)th station
            station = self.time_table.l[i][0]
            station = get_station[station]
            ll = self.time_table.l[i][1].split('.')
            travel_time = datetime.timedelta(hours=int(ll[0]),minutes=int(ll[1])).total_seconds()

            if(i==0):
                print('env.now, strt_time - ',convert(env.now) ,',', convert(travel_time))
                if(travel_time <= env.now):
                    print('train started before our simulation time')
                    return
                else:
                    t = travel_time - env.now
                    if(t<0):
                        t += 86400
                    yield self.env.timeout(t)
                    # print(f'Train {self.train_id} originates from station {station.station_id}')
                    prev_time = travel_time
            
            service_time = self.time_table.l[i][2]
            # yield self.env.timeout(service_time)
            with station.segment.resource.request() as req:
                yield req
                #print(f'station {station.station_id} has {station.segment.resource.count} trains at time {self.env.now}.')
                print(f'Train {self.train_id} starts service at station {station.station_id} at {convert(self.env.now)}.')
                yield self.env.timeout(service_time)
                print(f'Train {self.train_id} leaves station {station.station_id} at {convert(self.env.now)}.')
            #print(f'station {station.station_id} has {station.segment.resource.count} trains at time {self.env.now}.')

        #code assuming no segments between stations
            # yield self.env.timeout(travel_time)
            # if(i==0):
            #     print(f'Train {self.train_id} originates from station {station.station_id} at {convert(self.env.now)}.')
            # else:
            #     print(f'Train {self.train_id} leaves from station {station.station_id} at {convert(self.env.now)}.')
            lll = self.time_table.l[i+1][1].split('.')
            next_time = datetime.timedelta(hours=int(lll[0]),minutes=int(lll[1])).total_seconds()
            travel_time = next_time - prev_time- service_time
            prev_time = next_time
            if(travel_time<0):
                travel_time += 86400
                prev_time += 86400
            yield self.env.timeout(travel_time)

            
            

            # prev_req = req

        #code assuming segments between stations
            # next_station = self.time_table.l[i+1][0]
            # next_station = get_station[next_station]
            # seg_name = station.station_id+"_"+next_station.station_id
            # seg_list = segments[seg_name]
            # seg_time = self.time_table.l[i][3]
            # n = len(seg_list)
            # for j in range(n):
            #     seg = seg_list[seg_name+"_"+str(j)]
            #     if(seg.signal == 'green'):
            #         with seg.resource.request() as req:
            #             yield req
            #             seg.train_present = self.train_id
            #             seg.set_signal('red')
            #             if(j==0):
            #                 station.segment.set_signal('green')
            #                 station.segment.resource.release()


        ####
        # last_station = self.time_table.l[-1][0]
        # yield self.env.timeout(self.time_table.l[-1][1])
        # last_station_name = get_station[last_station]
        # print(f'Train {self.train_id} arrives at Last station {last_station_name.station_id} at {self.env.now}.')


env = simpy.Environment()

t = timedelta(hours=0, minutes=0, seconds=5)
env.run(until=t.total_seconds())
print('time before running the simulation :',convert(env.now))

#all times are departure except the last station for which time is arrival
# l[i] = (stn_id,departure_time,service_time,dist_prev_stn)
schedules = []
for t in timetable:
    schedules.append(timetable(t))


# l2 = [('MAS', '06.25', 0.0, 0), ('ABU', '06.46', 60.0, 16), ('TRL', '07.10', 120.0, 26), ('AJJ', '07.35', 120.0, 27), ('TRT', '07.55', 60.0, 14), ('EKM', '08.15', 120.0, 18), ('PUT', '08.30', 60.0, 14), ('RU', '09.05', 600.0, 26), ('TPTY', '09.40', 0.0, 10)]
# l1 = [('MAS', '07.40', 0.0, 0), ('AJJ', '08.40', 120.0, 69), ('SHU', '09.00', 120.0, 21), ('WJR', '09.15', 120.0, 15), ('KPD', '09.40', 120.0, 25), ('AB', '10.20', 120.0, 52), ('VN', '10.35', 120.0, 16), ('JTJ', '11.00', 120.0, 16), ('KPN', '11.40', 60.0, 43), ('BWT', '12.11', 120.0, 35), ('KJM', '12.52', 120.0, 56), ('BNC', '13.10', 120.0, 10), ('SBC', '13.40', 0.0, 4)]

# schedules.append(l1)
# schedules.append(l2)

for l in schedules:
    for i in range(len(l)):
        s = l[i][0]
        if s not in get_station:
            get_station[s] = station(env,s,1)

# 
for i in range(len(schedules)):
    t = timetable(env,schedules[i], schedules[i][0])
    t.make_schedule()



# brind_expr_schedule = timetable(env,l1, 'MAS')
# brind_expr_schedule.make_schedule()
# sapthagiri_schedule = timetable(env,l2, 'MAS')
# sapthagiri_schedule.make_schedule()

# for i in range(len(schedules)):
#     s = timetable(env,schedules[i], schedules[i][0])
#     s.make_schedule()


train_list = []
for t in timetable:
    train_list.append(Train(env, t, timetable[t]))
# train1 = Train(env, 'brindavan express', brind_expr_schedule)
# train2 = Train(env, 'sapthagiri express', sapthagiri_schedule)
# train_list.append(train1)
# train_list.append(train2)
for i in train_list:
    env.process(i.run())



# print("ji")
sim_time =  timedelta(hours=24, minutes=0, seconds=0)
env.run(until=sim_time.total_seconds())
print('time after completion of the simulation :', convert(env.now))
#printing all elements in the segments dictionary
# for i in segments:
#     print(i)
#     print(len(segments[i]))

# print(threading.active_count())




IndentationError: expected an indented block after 'for' statement on line 236 (1645252121.py, line 237)